<h1 style='text-align: center'>SQL Queries</h1>

## More Practice

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [5]:
# How many tables are in the database?

pd.read_sql_query('''
SELECT count(*)
FROM sqlite_master
WHERE type = 'table'
''', conn)

,count(*)
0,11


In [14]:
# What's the primary key of each table?
names_df = pd.read_sql_query('''
SELECT name
FROM sqlite_master
WHERE type = 'table'
''', conn)

for name in names_df.values:
    print(name[0])
    print(pd.read_sql_query('''
            PRAGMA table_info('{table_name}')
            '''.format(table_name=name[0]), conn))

Album
   cid      name           type  notnull dflt_value  pk
0    0   AlbumId        INTEGER        1       None   1
1    1     Title  NVARCHAR(160)        1       None   0
2    2  ArtistId        INTEGER        1       None   0
Artist
   cid      name           type  notnull dflt_value  pk
0    0  ArtistId        INTEGER        1       None   1
1    1      Name  NVARCHAR(120)        0       None   0
Customer
    cid          name          type  notnull dflt_value  pk
0     0    CustomerId       INTEGER        1       None   1
1     1     FirstName  NVARCHAR(40)        1       None   0
2     2      LastName  NVARCHAR(20)        1       None   0
3     3       Company  NVARCHAR(80)        0       None   0
4     4       Address  NVARCHAR(70)        0       None   0
5     5          City  NVARCHAR(40)        0       None   0
6     6         State  NVARCHAR(40)        0       None   0
7     7       Country  NVARCHAR(40)        0       None   0
8     8    PostalCode  NVARCHAR(10)        0  

In [18]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"

pd.read_sql_query('''PRAGMA foreign_key_list('PlaylistTrack')''', conn)


,id,seq,table,from,to,on_update,on_delete,match
0,0,0,Track,TrackId,TrackId,NO ACTION,NO ACTION,NONE
1,1,0,Playlist,PlaylistId,PlaylistId,NO ACTION,NO ACTION,NONE


In [22]:
# What are the max and min dates in the Invoice table?
pd.read_sql_query('''
SELECT max(InvoiceDate) AS max_date,
       min(InvoiceDate) AS min_date
FROM Invoice
''', conn)

,max_date,min_date
0,2013-12-22 00:00:00,2009-01-01 00:00:00


In [ ]:
# What tables would you need to answer "what is your most popular track?"
- name of track (Track)
- InvoiceLine
- Artist
- Album

In [ ]:
# What values from each table?
- Track.Name
- InvoiceLine.TrackID
- Track.ArtistID
- Album.AlbumId
- Artist.Name

In [39]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

pd.read_sql_query('''
SELECT
  t.Name AS track_name,
  ar.Name AS artist_name
FROM Track t
JOIN InvoiceLine il USING (TrackId)
JOIN Album al ON t.AlbumId = al.AlbumId
JOIN Artist ar ON al.ArtistId = ar.ArtistId

GROUP BY 1, 2
ORDER BY count(*) DESC

LIMIT 1
''', conn)

,track_name,artist_name
0,The Trooper,Iron Maiden


## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_matts
FROM student_courses
WHERE student_name = "Matt"
GROUP BY 1
HAVING COUNT(student_name) >= 3;
```

In [44]:
import sqlite3

conn_sqlite = sqlite3.connect('data/tutorial.db')
cursor_sqlite = conn_sqlite.cursor()

In [ ]:
!pip install pymysql

In [43]:
import pymysql

conn_mysql = pymysql.connect(host="fisdemo010620.c1doesqrid0e.us-east-1.rds.amazonaws.com",
                             user='fis_student',
                             password='SuperSecurePassword',
                             db='fis')
cursor_mysql = conn_mysql.cursor()

In [49]:
cursor_mysql.execute('select * from students')
columns = [x[0] for x in cursor_mysql.description]

In [46]:
columns

['name',
 'birthdate',
 'siblings',
 'birth_place',
 'years_in_nyc',
 'favorite_food']

In [50]:
cursor_sqlite.execute('select * from students')
columns = [x[0] for x in cursor_sqlite.description]
columns

['name',
 'birthdate',
 'siblings',
 'birth_place',
 'years_in_nyc',
 'favorite_food']

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [52]:
pd.read_sql_query('select name from students;', conn_mysql)

,name
0,Sean Abu Wilson
1,David Miller
2,Abhijeet Kamble
3,Samantha Jackson
4,Anmol Srivats
5,Ran Tokman
6,Amy Li
7,Florencia Leoni
8,Austin Krause
9,Natalie Overchuk


2. Which student has the most siblings?

In [58]:
pd.read_sql_query('''
SELECT
  name, siblings
FROM students
WHERE siblings = (SELECT max(siblings) FROM students)
''', conn_mysql)

,name,siblings
0,Florencia Leoni,4
1,Mohamad Eldebek,4
2,Menachi Korn,4
3,Miguel Peña,4


3. How many students are only children?

In [59]:
pd.read_sql_query('''
SELECT
  name, siblings
FROM students
WHERE siblings = 0
''', conn_mysql)

,name,siblings
0,Abhijeet Kamble,0
1,Anmol Srivats,0
2,Adam Dick,0


4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [ ]:
c.execute('''
-- YOUR CODE HERE
''').fetchall()

5. How many students are native New Yorkers?

In [ ]:
c.execute('''
-- YOUR CODE HERE
''').fetchall()

6. Do any two students have the same favorite food?

In [66]:
pd.read_sql_query("""SELECT favorite_food, count(favorite_food)
FROM students
GROUP BY 1
HAVING count(favorite_food) > 1
""", conn_mysql)

,favorite_food,count(favorite_food)
0,pizza,2
1,steak,2


## More Questions

What are the favorite foods of this classroom?

In [ ]:
c.execute("""
-- YOUR CODE HERE
""").fetchall()

7. Which student was born closest to the cohort's graduation date?

In [ ]:
c.execute('''
-- YOUR CODE HERE
''').fetchall() 